# Related Work

## ImMens

### Summary
ImMens contribution is limiting the cube size by the viewport instead of data limitations

## Nano Cubes

### Contributions

These are the contributions specified in the paper.

1. a novel data structure that improves on the current state of the art data cube technology to enable real-time exploratory visualization of multidimensional, spatiotemporal datasets;
1. algorithms to query the nanocube and build linked and brushable visual encodings commonly found in visualization systems; and
1. case studies highlighting the strengths and weaknesses of our technique, together with experiments to measure its utilization of space, time, and network bandwidth.

### Features

1. Data bound by display size (in support of visualization).
    1. In reality, since the primary visualization is spatial-based (first dimension is spatial), the resolution of data is mapped to the map tile size.
1. Supports any kind of summary that is algebraic with weighted sums and subtractions such as moment statistics.
1. Links are shared across dimensions and within dimensions to save space.
1. Support three dimension types in order:
    1. spatial with potentially multiple levels of fidelity
    2. categorical with possibly multiple, independent categories
    3. temporal
1. Temporal data is supported using summed area tables, which store cumulative counts progressing in time such that the count from one bin to a later bin (i.e. continuous time range) is the difference in bin values.  To find count only requires two binary searches, one to find the lower and upper bin, and a subtraction.

### Notes (Limitations)
1. The first dimension is always spatial
1. The content of the last dimension is always a time-series.
1. Returned data aligns to the minimal set of spatial labels that cover the query region. Therefore, returned results are from a spatial region either equal to or larger than the query region according to the spatial grid size.
1. Results processing speed in nano-cubes are bound by the tree structure.
1. Does not support arbitrarily shaped query regions (they claim it's possible, just more complex).  The queries now return more than the query region when the query region does not exactly overlap with their grid.
1. Summed-area tables used for time-series are fast for lookups but are inherently limited.  They either take up a larger amout of space across the board and/or are limited to a sum that is less than or equal to the largest value storable in the data size used for the sum table.
1. Timing and memory use is based on the cube storing count only (no algebraic or moment statistics).
    1. Different measures are claimed as supported but it is apparent that is not the case without changing the time-series storage of measures technique. In turn, another structure supporting those other features likely increases memory consumption beyond a 'high-end laptop' RAM (defined as 16GB) as well as increasing time to query.

### Benchmarks

#### Memory

The table below provides insight in to the memory consumption of nanocubes built with various datasets.

dataset|objects (N)|memory|build|size|sharing|keys (&#124;K&#124;)|&#124;K&#124;|schema
:-:|-:|-:|-:|-:|-:|-:|-:|:-
brightkite| 4.5 M| 1.6 GB|3.50 m|149.0 M| 3.00x| 3.5 M| 2<sup>74</sup>|lat(25), lon(25), time(16), weekday(3), hour(5)
customer tix| 7.8 M|2.5 GB|8.47 m|213.0 M| 2.93x| 7.8 M| 2<sup>69</sup>|lat(25), lon(25), time(16), type(3)
flights|121.0 M|2.3 GB|31.13 m| 274.0 M| 16.50x| 43.3 M| 2<sup>75</sup>|lat(25), lon(25), time(16), carrier(5), delay(4)
twitter-small| 210.0 M|10.2 GB|1.23 h|1.2 B|3.72x|116.0 M|2<sup>53</sup>|lat(17), lon(17), time(16), device(3)
twitter|210.0 M|46.4 GB| 5.87 h|5.2 B|4.00x|136.0 M|2<sup>60</sup>|lat(17), lon(17), time(16), lang(5), device(3), app(2)
splom-10| 1.0 B|4.3 MB|4.13 h|51.2 K|5.67x|7.4 K|2<sup>20</sup>|d1(4), d2(4), d3(4), d4(4), d5(4)
splom-50| 1.0 B|166.0 MB|4.72 h|8.8 M|16.00x|1.9 M|2<sup>30</sup>|d1(6), d2(6), d3(6), d4(6), d5(6)
cdrs|1.0 B|3.6 GB|3.08 h|271.0 M|18.60x|96.3 M|2<sup>69</sup>|lat(25), lon(25), time(16), duration(3)

* **objects(N)** is the number of input rows.
* **memory** is the size of the nanocube in memory
* **build** is the nanocube build time
* **size** is the number of nodes in the nanocube
* **sharing** indicates the data reduction from using shared nodes
* **schema** The numbers in parentheses on the schema column denote the number of bits necessary to refer to a value of that dimension

#### Timing

Nanocubes claims that queries are consistent across all experiments with mean query time of 830 microseconds (max = 12 milliseconds).  Average data returned from each query is 5KB (50KB max).  Geographical tiles are the biggest consumers of data.  The table below gives some insight into the data with more described in the following sections.

#### Comparison

Nanocubes provides comparison with imMens.

Technique    |             Space                  |    Query Time    | Constraints
:-----------:|:----------------------------------:|:----------------:|:-----------------------------:
Datavore [18]| O(&#124;K&#124;log2&#124;K*&#124;) | O(&#124;K&#124;) | &#124;K&#124; ≤ Main Mem.
imMens [21]  | O(&#124;K*&#124;)                  |      O(1)        | &#124;K*&#124; ≤ GPU Mem.
Nanocubes    | O(&#124;f(Ka)&#124;)               |      O(1)        | &#124;f(Ka)&#124; ≤ Main Mem.


### Data Details
#### Twitter Data
1. November 2011 - June 2012
1. 210 million tweets from public feed
1. Approximately 1 million tweets per day
1. Parameters
    1. Longitude
    1. Latitude
    1. Time
    1. Client Application (Categorical: 4 Options)
    1. Type of Device (Categorical: 5 Options)
    1. Language of Tweet (Categorical: 15 Options)
    
#### Airline Commercial Flights History
1. All commercial flights 1987 - 2008
1. 120 million flights
1. Parameters
    1. Scheduled departure time (Scheduled arrival time available but not used)
    1. Actual departure time (Actual arrival time available but not used)
    1. Origin airport (Destination airport available but not used)
    1. Airline
    1. Other non-described fields
1. Only the origin airport, which provides the geospatial information, is considered in the index

#### Call Detail Records
1. 1 month of data in July 2010
1. 1 billion records
1. Data was anonymized and aggregated
1. Parameters
    1. Position of first cell tower (All towers used is available in the data)
    1. Time of call
    1. Duration

#### Location-Based Social Networks
1. Brightkite checkins (site is defunct)
1. April 2008 - October 2010
1. Parameters
    1. Latitude, Longitude
    1. Time
    1. Added repetition of hour and day as categorical data

#### Customer Tickets
1. Customer interactions with service provider
1. 8 million rows
1. 2.5 years
1. Parameters
    1. Latitude, Longitude
    1. Time
    1. Report type
1. Data is anonymized

#### SPLOM
1. Collection of synthetic datasets
1. Each dataset has 5 dimensions
1. Used as the 1 billion record dataset

## Bandlimited OLAP Cube (BLOC)

### Summary
From LDAV, which generally convenes immediately prior to VisWeek, BLOCs use signal processing methods such as Fourier transform to cube data.  I am still studying this paper.

### Note
After discussing with Dr. Zhao, it was determined that this paper is solving a slightly different problem and can be omitted.

# Street Cube
There are two variations of the Street Cube.  The street cube maps the (x, y) spatial dimension (e.g. lon, lat) to a geospatial feature.  Other dimensions are filled in as needed based on the analysis.

## Street-Taxi
This cube is intended to allow analysis of both taxis and the streets they travel.  Some example queries:

1. Find the average speed of a given street during rush hours (0600-0900 & 1600-1900)
1. Find the average speed a taxi drives on streets within a city district
1. Find, on average, the fastest traveled road in a city.
1. Combinations of the various query aspects between the above.
    1. Various measures are applicable at the cost of whatever aggregate information is needed to combine results.  For example, average speed requires both the mean in the aggregate as well as the count.  A median would require an entire list of values.
    1. Aspects related to zero or more taxis
    1. Aspects related to zero or more streets
    1. Aspects related to zero or more times.

## Origin-Destination
This cube provides linked data so that origin and destination pairs can be queried.  Some example queries:

1. Find all pickup locations in a city district during a given time.
1. Find the drop-off locations for pickups in a given area.
1. Find times during the day when pickups are most common
1. Combinastions of the various queries
1. Possible measures
    1. Trip time between origin and destination
    1. Trip distaince between origin and destination